In [39]:
%load_ext autoreload
%autoreload 2

import pandas as pd

df = pd.read_table("https://raw.githubusercontent.com/eliabisconti/haspeede/master/Dataset/haspeede2_dev_taskAB.tsv")

df_test = pd.read_table("https://raw.githubusercontent.com/collab-uniba/HateSpeechDetection_ITA/70d0b4240bb373c0e0a10f8a09abdc4d91f1b98e/Test%20sui%20Tweet/Train%20e%20test%20set%20UmBERTo/haspeede2_reference_taskAB-tweets.csv", header=None, names=["id", "text", "hs", "stereotype"])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
from sklearn.model_selection import train_test_split


train_df, dev_df = train_test_split(
    df, test_size=0.18, random_state=2022)

train_df.shape, dev_df.shape, df_test.shape

((5606, 4), (1231, 4), (1263, 4))

In [41]:
train_df.columns = [t.strip() for t in train_df.columns]
dev_df.columns = [t.strip() for t in dev_df.columns]
df_test.columns = [t.strip() for t in df_test.columns]

train_df.columns, dev_df.columns, df_test.columns

(Index(['id', 'text', 'hs', 'stereotype'], dtype='object'),
 Index(['id', 'text', 'hs', 'stereotype'], dtype='object'),
 Index(['id', 'text', 'hs', 'stereotype'], dtype='object'))

In [42]:
from datasets import DatasetDict, Dataset, Value, Features, ClassLabel

features = Features({
    'id': Value('string'),
    'text': Value('string'),
    'hs': ClassLabel(num_classes=2, names=["non hateful", "hateful"]),
    "stereotype": ClassLabel(num_classes=2, names=["non stereotype", "stereotype"]),
})

train = Dataset.from_pandas(train_df, features=features, preserve_index=False)
dev = Dataset.from_pandas(dev_df, features=features, preserve_index=False)
test = Dataset.from_pandas(df_test, features=features, preserve_index=False)

ds = DatasetDict(
    train=train,
    dev=dev,
    test=test
)

In [43]:
ds.push_to_hub("pysentimiento/it_haspeede", private=True)

Pushing split train to the Hub.
Resuming upload of the dataset shards.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split dev to the Hub.
Resuming upload of the dataset shards.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split test to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

## Check Haspeede 2020

In [10]:
from datasets import load_dataset

ds = load_dataset("pysentimiento/it_haspeede")

ds

Using custom data configuration pysentimiento--it_haspeede-213526540a343f67
Found cached dataset parquet (/users/jmperez/.cache/huggingface/datasets/pysentimiento___parquet/pysentimiento--it_haspeede-213526540a343f67/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    dev: Dataset({
        features: ['id', 'text', 'hs', 'stereotype'],
        num_rows: 1231
    })
    train: Dataset({
        features: ['id', 'text', 'hs', 'stereotype'],
        num_rows: 5606
    })
})

In [12]:
df = pd.read_csv("../../data/hate/haspeede2020.csv")

df["set"].value_counts()

training    6840
test_tw     1263
test_nw      500
Name: set, dtype: int64

In [11]:
text_ds = ds["train"]["text"] + ds["dev"]["text"]

len(text_ds)

6837

In [19]:
df

,id_source,text_original,set,dataset
0,_1039886625994694656,L'immigrato che non piace alla sinistra razzis...,training,HaSpeeDe2020
1,_1039892173595062273,"Educazione stradale, i Vigili in Pensione sezi...",training,HaSpeeDe2020
2,_1039910346432147456,@AFoccardi @SoniaGrotto ... ma tutti i psicola...,training,HaSpeeDe2020
3,_1039913608287801346,Un grave episodio che evidenzia i danni dell'#...,training,HaSpeeDe2020
4,_1039916875226988544,@GiorgiaMeloni @FratellidItaIia A Gio' visto c...,training,HaSpeeDe2020
...,...,...,...,...
8598,_titoli-cri-597,"L'Imam di Catania contro i clandestini: ""Basta...",test_nw,HaSpeeDe2020
8599,_titoli-cri-62,Sapete cosa voglio fare agli immigrati violent...,test_nw,HaSpeeDe2020
8600,_titoli-cri-64,Sapete quanti soldi diamo agli immigrati e qua...,test_nw,HaSpeeDe2020
8601,_titoli-cri-9,Così i profughi ci svuotano i negozi a Pordenone,test_nw,HaSpeeDe2020


In [16]:
training_df = df[df["set"] == "train"]

In [17]:
len(set(training_df["text_original"]) - set(text_ds))

0

In [21]:
test_df = df[df["set"] == "test_tw"]

In [24]:
test_df["text_original"].iloc[0]

'@LaStampaTV @LaStampa i musulmano ammazzano tutti quelli che per il loro fumoso cervello sono "infedeli". i nostri terroristi istituzionali ci obbligano ad accogliere e mantenere i nostri assassini'

In [32]:
pd.options.display.max_colwidth = 100

df_test

,text,hs,stereotype
id,,,
11834,@user A me pare una scelta politica suicida puntare tutto su una battaglia sicuramente perdente ...,1,0
12113,"@user e' un perfetto musulmano!!! chi dice il contrario mente, applica alla lettera la sura ""la ...",1,1
11770,Mai Tg e i giornaloni hanno parlato di questa ennesima porcata fatta da immigrati ?? ⤵️⤵️⤵️⤵️ S...,1,1
11937,@user Ipocriti farabutti. Fanno morire i terremotati per I Bastardi clandestini immigrati schifosi.,1,1
11870,@user @user @user L'IMMIGRAZIONE C'E' STATO UN CALO FISIOLOGICO QUANTI RINPATRII HA FATTO ? CI S...,1,1
...,...,...,...
10091,"Il Razzismo gli viene inculcato sin da bambini dal corano e dall'islam Cani, Maiali. Cristiani e...",1,1
10046,"PERCHÉ DI QUESTO ORRENDO STUPRO NON DI PARLA? INFORMAZIONE CORROTTA,LA SINISTRA CONTINUA A COPRI...",1,1
10377,"FERMIAMO L'IMPOSTORE. Forse saranno necesarie altre CROCIATE, visto che il fiume di fango non si...",1,1


In [ ]:
df_t